In [1]:
import torch
import pandas as pd
import shapely
import numpy as np
import sys
import torch
import argparse
import random
import math
import os

from sklearn.preprocessing import OneHotEncoder
from scipy.stats import stats
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler

In [2]:
# read file
# use the same data as for regression model
#data = pd.read_pickle('../data/thread_0_0_999.pkl')
data = pd.read_csv('regression.csv', index_col=0)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,693,694,695,696,697,698,699,700,701,702
92600-1005_1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.51,1838.3,0.831534
92600-1005_10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.44,2101.3,0.840493
92600-1005_13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.33,2652.4,0.882263
92600-1005_14,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.71,2344.3,0.875813
92600-1005_15,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.85,1897.6,0.827455


In [33]:
df = data.copy()

In [38]:
data_x = df.iloc[:, :700].values.reshape((-1, 50, 14))

In [86]:
data_y = df.iloc[:, 700:]

In [87]:
Interquartile0_25 = np.array(data_y.quantile(0.25))
Interquartile0_75 = np.array(data_y.quantile(0.75))
IQR = Interquartile0_75 - Interquartile0_25

# Median for each features
median = np.array(data_y.median())


# sigmoid transformation
numeric_features = (1+np.exp((-data_y + median)/(0.5*IQR)))**(-1)
data_y = numeric_features.sum(axis=1)

In [93]:
data_y = data_y.values.reshape(-1, 1)

In [94]:
data_y.shape

(15302, 1)

In [89]:
data_x[:,1,:].shape

torch.Size([15302, 14])

In [95]:
# save to the local system
np.save('../data/mlp_train_x', data_x[:12000, :])
np.save('../data/mlp_train_y', data_y[:12000, :])
np.save('../data/mlp_test_x', data_x[12000:, :])
np.save('../data/mlp_test_y', data_y[12000:, :])

## build the mlp model

In [105]:
# bild the model, loss and data class
class TS_mlp(torch.nn.Module):
    """
    scores for each piece
    input:
        tensor size of (batch_size, seq_len, num_dim)
    output:
        tensor size of (batch_size, seq_len)
    """
    def __init__(self):
        super(TS_mlp, self).__init__()
        #change the structure of the network
        num_inp = 8
        num_hidden = 64
        self.mlp1 = torch.nn.Sequential(
                torch.nn.Linear(14, 36),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(36),
                torch.nn.Linear(36, 8),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(8)
        )
        self.mlp2 = torch.nn.Sequential(
                torch.nn.Linear(400, 64),
                torch.nn.Dropout(),
                torch.nn.ReLU(),
                torch.nn.Linear(64, 1)
                )

    def forward(self, inp):
        # inp of shape (batch, num_part, num_dim)
        batch_size, num_part, num_dim = inp.shape
        out_mlp1 = []
        for i in range(num_part):
            out_mlp1.append(self.mlp1(inp[:, i, :]))
        inp_mlp2 = torch.cat(out_mlp1, dim = 1)
        out = self.mlp2(inp_mlp2)
        return out.squeeze()
    
class PDLoss(torch.nn.Module):
    def __init__(self, p = 2):
        super(PDLoss, self).__init__()
        self.pd = torch.nn.PairwiseDistance(p)

    def forward(self, o, t):
        # out: (batch_size, 1)
        out = self.pd(o, t)
        return out.mean()

class Data:
    """
    data class for TS_rnn
    """
    def __init__(self, x, y):
        self.data = {}
        self.data['train_x'] = self.add_file(x)
        self.data['train_y'] = self.add_file(y)[:, -1] # use the first metric tempately
        assert(len(self.data['train_x']) == len(self.data['train_y']))
        self.len = len(self.data['train_x'])

    def add_file(self, path):
        return torch.from_numpy(np.load(path)).type(torch.FloatTensor)

    def add_scores(self, path):
        return torch.FloatTensor([float(li.rstrip('\n')) for li in open(path)])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return (self.data['train_x'][index],
                self.data['train_y'][index])
# write the test function
def test_model(dl_test, model, loss):
    model.eval()
    test_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_test):
        counter += 1
        # codes to be changed
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        test_loss += lo.data
    return test_loss/counter
def significant_test(dl_test, model, loss):
    model.eval()
    test_loss = 0
    counter = 0
    hit = 0
    miss = 0
    for batch_idx, dat in enumerate(dl_test):
        counter += 1
        # codes to be changed
        inp, target = dat
        out = model(inp)
        if len(inp) > 10:
            _, top_target = torch.topk(target, 1)
            _, top_predict = torch.topk(out, 10)
            if top_target in top_predict:
                hit += 1
            else:
                miss += 1
    return hit * 1.0/(hit + miss)

In [106]:
# train the model
train_x = '../data/mlp_train_x.npy'
train_y = '../data/mlp_train_y.npy'
test_x = '../data/mlp_test_x.npy'
test_y = '../data/mlp_test_y.npy'
num_epochs = 100
batch_size = 30
verbose = True
# set models and loss
model = TS_mlp()
#loss = PDLoss()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
# set the scheduler
lamb1 = lambda x: .1**(x//30)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda = lamb1)
# loda data
train = Data(train_x, train_y)
test = Data(test_x, test_y)
dl_train = DataLoader(train, batch_size = batch_size, shuffle = True)
dl_test = DataLoader(test, batch_size = batch_size, shuffle = True)
# train the model
for epoch in range(num_epochs):
    scheduler.step()
    model.train()
    train_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_train):
        counter += 1
        # train the model
        optimizer.zero_grad()
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        lo.backward()
        optimizer.step()
        train_loss += lo.data
        if verbose:
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * batch_size,
                    len(train),
                    100.*batch_idx*batch_size/len(train),
                    lo.data
                    ))
    test_lo = test_model(dl_test, model, loss)
    hit_rate = significant_test(dl_test, model, loss)
    if verbose:
        # train loss
        print('====> Epoch: {} Average train loss: {:.4f}'.format(
            epoch,
            train_loss/counter
            ))
        # test loss
        print('====> Epoch: {} Average test loss: {:.4f}'.format(
            epoch,
            test_lo
            ))
        # significant test
        print('====> Epoch: {} Average hit rate in 10 candidate: {: .4f}'.format(
            epoch,
            hit_rate
        ))

Train Epoch: 0 [0/12000 (0%)]	Loss: 1.352626
Train Epoch: 0 [300/12000 (2%)]	Loss: 1.331267
Train Epoch: 0 [600/12000 (5%)]	Loss: 1.010649
Train Epoch: 0 [900/12000 (8%)]	Loss: 0.868061
Train Epoch: 0 [1200/12000 (10%)]	Loss: 0.589279
Train Epoch: 0 [1500/12000 (12%)]	Loss: 0.387036
Train Epoch: 0 [1800/12000 (15%)]	Loss: 0.502994
Train Epoch: 0 [2100/12000 (18%)]	Loss: 0.471911
Train Epoch: 0 [2400/12000 (20%)]	Loss: 0.443900
Train Epoch: 0 [2700/12000 (22%)]	Loss: 0.510564
Train Epoch: 0 [3000/12000 (25%)]	Loss: 0.465224
Train Epoch: 0 [3300/12000 (28%)]	Loss: 0.507298
Train Epoch: 0 [3600/12000 (30%)]	Loss: 0.476584
Train Epoch: 0 [3900/12000 (32%)]	Loss: 0.595696
Train Epoch: 0 [4200/12000 (35%)]	Loss: 0.536561
Train Epoch: 0 [4500/12000 (38%)]	Loss: 0.566998
Train Epoch: 0 [4800/12000 (40%)]	Loss: 0.482891
Train Epoch: 0 [5100/12000 (42%)]	Loss: 0.518418
Train Epoch: 0 [5400/12000 (45%)]	Loss: 0.492202
Train Epoch: 0 [5700/12000 (48%)]	Loss: 0.497632
Train Epoch: 0 [6000/12000 (50

Train Epoch: 3 [11700/12000 (98%)]	Loss: 0.430702
====> Epoch: 3 Average train loss: 0.4462
====> Epoch: 3 Average test loss: 0.4295
====> Epoch: 3 Average hit rate in 10 candidate:  0.4364
Train Epoch: 4 [0/12000 (0%)]	Loss: 0.472075
Train Epoch: 4 [300/12000 (2%)]	Loss: 0.414430
Train Epoch: 4 [600/12000 (5%)]	Loss: 0.469693
Train Epoch: 4 [900/12000 (8%)]	Loss: 0.385444
Train Epoch: 4 [1200/12000 (10%)]	Loss: 0.415696
Train Epoch: 4 [1500/12000 (12%)]	Loss: 0.511211
Train Epoch: 4 [1800/12000 (15%)]	Loss: 0.374522
Train Epoch: 4 [2100/12000 (18%)]	Loss: 0.404812
Train Epoch: 4 [2400/12000 (20%)]	Loss: 0.406972
Train Epoch: 4 [2700/12000 (22%)]	Loss: 0.472708
Train Epoch: 4 [3000/12000 (25%)]	Loss: 0.355252
Train Epoch: 4 [3300/12000 (28%)]	Loss: 0.449413
Train Epoch: 4 [3600/12000 (30%)]	Loss: 0.412273
Train Epoch: 4 [3900/12000 (32%)]	Loss: 0.389150
Train Epoch: 4 [4200/12000 (35%)]	Loss: 0.427186
Train Epoch: 4 [4500/12000 (38%)]	Loss: 0.546202
Train Epoch: 4 [4800/12000 (40%)]	Lo

Train Epoch: 7 [10800/12000 (90%)]	Loss: 0.431664
Train Epoch: 7 [11100/12000 (92%)]	Loss: 0.442359
Train Epoch: 7 [11400/12000 (95%)]	Loss: 0.377095
Train Epoch: 7 [11700/12000 (98%)]	Loss: 0.341136
====> Epoch: 7 Average train loss: 0.4218
====> Epoch: 7 Average test loss: 0.4859
====> Epoch: 7 Average hit rate in 10 candidate:  0.4455
Train Epoch: 8 [0/12000 (0%)]	Loss: 0.406275
Train Epoch: 8 [300/12000 (2%)]	Loss: 0.507234
Train Epoch: 8 [600/12000 (5%)]	Loss: 0.529734
Train Epoch: 8 [900/12000 (8%)]	Loss: 0.384487
Train Epoch: 8 [1200/12000 (10%)]	Loss: 0.323787
Train Epoch: 8 [1500/12000 (12%)]	Loss: 0.458067
Train Epoch: 8 [1800/12000 (15%)]	Loss: 0.424299
Train Epoch: 8 [2100/12000 (18%)]	Loss: 0.390350
Train Epoch: 8 [2400/12000 (20%)]	Loss: 0.413516
Train Epoch: 8 [2700/12000 (22%)]	Loss: 0.477936
Train Epoch: 8 [3000/12000 (25%)]	Loss: 0.412915
Train Epoch: 8 [3300/12000 (28%)]	Loss: 0.472561
Train Epoch: 8 [3600/12000 (30%)]	Loss: 0.494767
Train Epoch: 8 [3900/12000 (32%)]

Train Epoch: 11 [9300/12000 (78%)]	Loss: 0.370853
Train Epoch: 11 [9600/12000 (80%)]	Loss: 0.480731
Train Epoch: 11 [9900/12000 (82%)]	Loss: 0.475687
Train Epoch: 11 [10200/12000 (85%)]	Loss: 0.332977
Train Epoch: 11 [10500/12000 (88%)]	Loss: 0.412661
Train Epoch: 11 [10800/12000 (90%)]	Loss: 0.320147
Train Epoch: 11 [11100/12000 (92%)]	Loss: 0.357679
Train Epoch: 11 [11400/12000 (95%)]	Loss: 0.412169
Train Epoch: 11 [11700/12000 (98%)]	Loss: 0.448965
====> Epoch: 11 Average train loss: 0.4088
====> Epoch: 11 Average test loss: 0.6121
====> Epoch: 11 Average hit rate in 10 candidate:  0.6091
Train Epoch: 12 [0/12000 (0%)]	Loss: 0.316623
Train Epoch: 12 [300/12000 (2%)]	Loss: 0.422551
Train Epoch: 12 [600/12000 (5%)]	Loss: 0.398782
Train Epoch: 12 [900/12000 (8%)]	Loss: 0.360300
Train Epoch: 12 [1200/12000 (10%)]	Loss: 0.404221
Train Epoch: 12 [1500/12000 (12%)]	Loss: 0.366237
Train Epoch: 12 [1800/12000 (15%)]	Loss: 0.361551
Train Epoch: 12 [2100/12000 (18%)]	Loss: 0.388219
Train Epoch

Train Epoch: 15 [7200/12000 (60%)]	Loss: 0.492061
Train Epoch: 15 [7500/12000 (62%)]	Loss: 0.432115
Train Epoch: 15 [7800/12000 (65%)]	Loss: 0.337286
Train Epoch: 15 [8100/12000 (68%)]	Loss: 0.381280
Train Epoch: 15 [8400/12000 (70%)]	Loss: 0.390922
Train Epoch: 15 [8700/12000 (72%)]	Loss: 0.385416
Train Epoch: 15 [9000/12000 (75%)]	Loss: 0.364321
Train Epoch: 15 [9300/12000 (78%)]	Loss: 0.320828
Train Epoch: 15 [9600/12000 (80%)]	Loss: 0.439639
Train Epoch: 15 [9900/12000 (82%)]	Loss: 0.420761
Train Epoch: 15 [10200/12000 (85%)]	Loss: 0.395306
Train Epoch: 15 [10500/12000 (88%)]	Loss: 0.480948
Train Epoch: 15 [10800/12000 (90%)]	Loss: 0.378478
Train Epoch: 15 [11100/12000 (92%)]	Loss: 0.355819
Train Epoch: 15 [11400/12000 (95%)]	Loss: 0.428052
Train Epoch: 15 [11700/12000 (98%)]	Loss: 0.392571
====> Epoch: 15 Average train loss: 0.3937
====> Epoch: 15 Average test loss: 0.4437
====> Epoch: 15 Average hit rate in 10 candidate:  0.4818
Train Epoch: 16 [0/12000 (0%)]	Loss: 0.483747
Train

Train Epoch: 19 [5100/12000 (42%)]	Loss: 0.355448
Train Epoch: 19 [5400/12000 (45%)]	Loss: 0.382572
Train Epoch: 19 [5700/12000 (48%)]	Loss: 0.376075
Train Epoch: 19 [6000/12000 (50%)]	Loss: 0.344718
Train Epoch: 19 [6300/12000 (52%)]	Loss: 0.396869
Train Epoch: 19 [6600/12000 (55%)]	Loss: 0.337061
Train Epoch: 19 [6900/12000 (58%)]	Loss: 0.442760
Train Epoch: 19 [7200/12000 (60%)]	Loss: 0.285799
Train Epoch: 19 [7500/12000 (62%)]	Loss: 0.376847
Train Epoch: 19 [7800/12000 (65%)]	Loss: 0.312374
Train Epoch: 19 [8100/12000 (68%)]	Loss: 0.379864
Train Epoch: 19 [8400/12000 (70%)]	Loss: 0.366348
Train Epoch: 19 [8700/12000 (72%)]	Loss: 0.372675
Train Epoch: 19 [9000/12000 (75%)]	Loss: 0.433851
Train Epoch: 19 [9300/12000 (78%)]	Loss: 0.338119
Train Epoch: 19 [9600/12000 (80%)]	Loss: 0.397533
Train Epoch: 19 [9900/12000 (82%)]	Loss: 0.406730
Train Epoch: 19 [10200/12000 (85%)]	Loss: 0.367216
Train Epoch: 19 [10500/12000 (88%)]	Loss: 0.355328
Train Epoch: 19 [10800/12000 (90%)]	Loss: 0.3551

KeyboardInterrupt: 

In [ ]:
# save model
torch.save(model, '../models/mlp_predict.pkl')

## build rnn model to predict final metric
copy from TS_rnn.ipynt

In [111]:
# bild the model, loss and data class
class TS_rnn(torch.nn.Module):
    """
    scores only for the whole task
    input:
        tensor size of (batch_size, seq_len, num_dim)
    output:
        tensor size of (batch_size)
    """
    def __init__(self):
        super(TS_rnn, self).__init__()
        #change the structure of the network
        num_inp = 14
        num_hidden = 64
        self.rnn = torch.nn.LSTM(input_size = num_inp, hidden_size = num_hidden, num_layers = 2)
        self.mlp = torch.nn.Sequential(
                torch.nn.Linear(num_hidden, 64),
                torch.nn.Dropout(),
                torch.nn.ReLU(),
                torch.nn.Linear(64, 1)
                )

    def forward(self, inp):
        # input of the rnn (seq_len, batch, input_size)
        data_in = torch.transpose(inp, 0, 1)
        # run rnn, it has two output
        out_rnn, _ = self.rnn(data_in)
        out_rnn = torch.transpose(out_rnn, 0, 1) # (batch_size, seq_len, num_dim)
        # only use the last output
        out_rnn = out_rnn[:, -1, :].squeeze()
        # rnn the mlp
        out = self.mlp(out_rnn)
        return out.squeeze()
    
class PDLoss(torch.nn.Module):
    def __init__(self, p = 2):
        super(PDLoss, self).__init__()
        self.pd = torch.nn.PairwiseDistance(p)

    def forward(self, o, t):
        # out: (batch_size, 1)
        out = self.pd(o, t)
        return out.mean()

class Data:
    """
    data class for TS_rnn
    """
    def __init__(self, x, y):
        self.data = {}
        self.data['train_x'] = self.add_file(x)
        self.data['train_y'] = self.add_file(y)[:, -1] # use the first metric tempately
        assert(len(self.data['train_x']) == len(self.data['train_y']))
        self.len = len(self.data['train_x'])

    def add_file(self, path):
        return torch.from_numpy(np.load(path)).type(torch.FloatTensor)

    def add_scores(self, path):
        return torch.FloatTensor([float(li.rstrip('\n')) for li in open(path)])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return (self.data['train_x'][index],
                self.data['train_y'][index])
    
def test_model(dl_test, model, loss):
    model.eval()
    test_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_test):
        counter += 1
        # codes to be changed
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        test_loss += lo.data
    return test_loss/counter

def significant_test(dl_test, model, loss):
    model.eval()
    test_loss = 0
    counter = 0
    hit = 0
    miss = 0
    for batch_idx, dat in enumerate(dl_test):
        counter += 1
        # codes to be changed
        inp, target = dat
        out = model(inp)
        if len(inp) > 10:
            _, top_target = torch.topk(target, 1)
            _, top_predict = torch.topk(out, 5)
            if top_target in top_predict:
                hit += 1
            else:
                miss += 1
    return hit * 1.0/(hit + miss)

In [112]:
# train the model
train_x = '../data/mlp_train_x.npy'
train_y = '../data/mlp_train_y.npy'
test_x = '../data/mlp_test_x.npy'
test_y = '../data/mlp_test_y.npy'
num_epochs = 100
batch_size = 30
verbose = True
# set models and loss
model = TS_rnn()
#loss = PDLoss()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
# set the scheduler
lamb1 = lambda x: .1**(x//30)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda = lamb1)
# loda data
train = Data(train_x, train_y)
test = Data(test_x, test_y)
dl_train = DataLoader(train, batch_size = batch_size, shuffle = True)
dl_test = DataLoader(test, batch_size = batch_size, shuffle = True)
# train the model
for epoch in range(num_epochs):
    scheduler.step()
    model.train()
    train_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_train):
        counter += 1
        # train the model
        optimizer.zero_grad()
        inp, target = dat
        out = model(inp)
        lo = loss(out, target.squeeze())
        lo.backward()
        optimizer.step()
        train_loss += lo.data
        if verbose:
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * batch_size,
                    len(train),
                    100.*batch_idx*batch_size/len(train),
                    lo.data
                    ))
    test_lo = test_model(dl_test, model, loss)
    hit_rate = significant_test(dl_test, model, loss)
    if verbose:
        # train loss
        print('====> Epoch: {} Average train loss: {:.4f}'.format(
            epoch,
            train_loss/counter
            ))
        # test loss
        print('====> Epoch: {} Average test loss: {:.4f}'.format(
            epoch,
            test_lo
            ))
        # significant test
        print('====> Epoch: {} Average hit rate in 10 candidate: {: .4f}'.format(
            epoch,
            hit_rate
        ))

Train Epoch: 0 [0/12000 (0%)]	Loss: 1.582689
Train Epoch: 0 [300/12000 (2%)]	Loss: 1.249646
Train Epoch: 0 [600/12000 (5%)]	Loss: 0.444173
Train Epoch: 0 [900/12000 (8%)]	Loss: 0.570139
Train Epoch: 0 [1200/12000 (10%)]	Loss: 0.509248
Train Epoch: 0 [1500/12000 (12%)]	Loss: 0.557792
Train Epoch: 0 [1800/12000 (15%)]	Loss: 0.542035
Train Epoch: 0 [2100/12000 (18%)]	Loss: 0.517898
Train Epoch: 0 [2400/12000 (20%)]	Loss: 0.477862
Train Epoch: 0 [2700/12000 (22%)]	Loss: 0.574974
Train Epoch: 0 [3000/12000 (25%)]	Loss: 0.556975
Train Epoch: 0 [3300/12000 (28%)]	Loss: 0.602124
Train Epoch: 0 [3600/12000 (30%)]	Loss: 0.416932
Train Epoch: 0 [3900/12000 (32%)]	Loss: 0.564714
Train Epoch: 0 [4200/12000 (35%)]	Loss: 0.570002
Train Epoch: 0 [4500/12000 (38%)]	Loss: 0.492031
Train Epoch: 0 [4800/12000 (40%)]	Loss: 0.462375
Train Epoch: 0 [5100/12000 (42%)]	Loss: 0.434205
Train Epoch: 0 [5400/12000 (45%)]	Loss: 0.546060
Train Epoch: 0 [5700/12000 (48%)]	Loss: 0.587673
Train Epoch: 0 [6000/12000 (50

Train Epoch: 3 [11700/12000 (98%)]	Loss: 0.357273
====> Epoch: 3 Average train loss: 0.4841
====> Epoch: 3 Average test loss: 0.4456
====> Epoch: 3 Average hit rate in 10 candidate:  0.4727
Train Epoch: 4 [0/12000 (0%)]	Loss: 0.384259
Train Epoch: 4 [300/12000 (2%)]	Loss: 0.510546
Train Epoch: 4 [600/12000 (5%)]	Loss: 0.365669
Train Epoch: 4 [900/12000 (8%)]	Loss: 0.557983
Train Epoch: 4 [1200/12000 (10%)]	Loss: 0.449695
Train Epoch: 4 [1500/12000 (12%)]	Loss: 0.469766
Train Epoch: 4 [1800/12000 (15%)]	Loss: 0.502175
Train Epoch: 4 [2100/12000 (18%)]	Loss: 0.566892
Train Epoch: 4 [2400/12000 (20%)]	Loss: 0.567353
Train Epoch: 4 [2700/12000 (22%)]	Loss: 0.526581
Train Epoch: 4 [3000/12000 (25%)]	Loss: 0.477235
Train Epoch: 4 [3300/12000 (28%)]	Loss: 0.447594
Train Epoch: 4 [3600/12000 (30%)]	Loss: 0.464685
Train Epoch: 4 [3900/12000 (32%)]	Loss: 0.516131
Train Epoch: 4 [4200/12000 (35%)]	Loss: 0.495534
Train Epoch: 4 [4500/12000 (38%)]	Loss: 0.518253
Train Epoch: 4 [4800/12000 (40%)]	Lo

Train Epoch: 7 [10800/12000 (90%)]	Loss: 0.530900
Train Epoch: 7 [11100/12000 (92%)]	Loss: 0.421005
Train Epoch: 7 [11400/12000 (95%)]	Loss: 0.412513
Train Epoch: 7 [11700/12000 (98%)]	Loss: 0.453415
====> Epoch: 7 Average train loss: 0.4506
====> Epoch: 7 Average test loss: 0.4251
====> Epoch: 7 Average hit rate in 10 candidate:  0.4455
Train Epoch: 8 [0/12000 (0%)]	Loss: 0.391563
Train Epoch: 8 [300/12000 (2%)]	Loss: 0.451903
Train Epoch: 8 [600/12000 (5%)]	Loss: 0.400065
Train Epoch: 8 [900/12000 (8%)]	Loss: 0.530223
Train Epoch: 8 [1200/12000 (10%)]	Loss: 0.422301
Train Epoch: 8 [1500/12000 (12%)]	Loss: 0.422129
Train Epoch: 8 [1800/12000 (15%)]	Loss: 0.571843
Train Epoch: 8 [2100/12000 (18%)]	Loss: 0.421074
Train Epoch: 8 [2400/12000 (20%)]	Loss: 0.406145
Train Epoch: 8 [2700/12000 (22%)]	Loss: 0.352343
Train Epoch: 8 [3000/12000 (25%)]	Loss: 0.433275
Train Epoch: 8 [3300/12000 (28%)]	Loss: 0.474571
Train Epoch: 8 [3600/12000 (30%)]	Loss: 0.458893
Train Epoch: 8 [3900/12000 (32%)]

Train Epoch: 11 [9300/12000 (78%)]	Loss: 0.428028
Train Epoch: 11 [9600/12000 (80%)]	Loss: 0.370715
Train Epoch: 11 [9900/12000 (82%)]	Loss: 0.434077
Train Epoch: 11 [10200/12000 (85%)]	Loss: 0.395143
Train Epoch: 11 [10500/12000 (88%)]	Loss: 0.439189
Train Epoch: 11 [10800/12000 (90%)]	Loss: 0.441322
Train Epoch: 11 [11100/12000 (92%)]	Loss: 0.462985
Train Epoch: 11 [11400/12000 (95%)]	Loss: 0.493842
Train Epoch: 11 [11700/12000 (98%)]	Loss: 0.365086
====> Epoch: 11 Average train loss: 0.4256
====> Epoch: 11 Average test loss: 0.4134
====> Epoch: 11 Average hit rate in 10 candidate:  0.5545
Train Epoch: 12 [0/12000 (0%)]	Loss: 0.414049
Train Epoch: 12 [300/12000 (2%)]	Loss: 0.331814
Train Epoch: 12 [600/12000 (5%)]	Loss: 0.510127
Train Epoch: 12 [900/12000 (8%)]	Loss: 0.439973
Train Epoch: 12 [1200/12000 (10%)]	Loss: 0.457079
Train Epoch: 12 [1500/12000 (12%)]	Loss: 0.442149
Train Epoch: 12 [1800/12000 (15%)]	Loss: 0.421603
Train Epoch: 12 [2100/12000 (18%)]	Loss: 0.499953
Train Epoch

Train Epoch: 15 [7200/12000 (60%)]	Loss: 0.290521
Train Epoch: 15 [7500/12000 (62%)]	Loss: 0.361739
Train Epoch: 15 [7800/12000 (65%)]	Loss: 0.405049
Train Epoch: 15 [8100/12000 (68%)]	Loss: 0.410592
Train Epoch: 15 [8400/12000 (70%)]	Loss: 0.380458
Train Epoch: 15 [8700/12000 (72%)]	Loss: 0.462209
Train Epoch: 15 [9000/12000 (75%)]	Loss: 0.367838
Train Epoch: 15 [9300/12000 (78%)]	Loss: 0.475568
Train Epoch: 15 [9600/12000 (80%)]	Loss: 0.419411
Train Epoch: 15 [9900/12000 (82%)]	Loss: 0.359597
Train Epoch: 15 [10200/12000 (85%)]	Loss: 0.316466
Train Epoch: 15 [10500/12000 (88%)]	Loss: 0.378283
Train Epoch: 15 [10800/12000 (90%)]	Loss: 0.267864
Train Epoch: 15 [11100/12000 (92%)]	Loss: 0.397151
Train Epoch: 15 [11400/12000 (95%)]	Loss: 0.346282
Train Epoch: 15 [11700/12000 (98%)]	Loss: 0.512880
====> Epoch: 15 Average train loss: 0.4138
====> Epoch: 15 Average test loss: 0.4175
====> Epoch: 15 Average hit rate in 10 candidate:  0.5091
Train Epoch: 16 [0/12000 (0%)]	Loss: 0.422451
Train

...

Train Epoch: 80 [11100/12000 (92%)]	Loss: 0.244603
Train Epoch: 80 [11400/12000 (95%)]	Loss: 0.331767
Train Epoch: 80 [11700/12000 (98%)]	Loss: 0.363116
====> Epoch: 80 Average train loss: 0.3442
====> Epoch: 80 Average test loss: 0.4374
====> Epoch: 80 Average hit rate in 10 candidate:  0.4909
Train Epoch: 81 [0/12000 (0%)]	Loss: 0.431194
Train Epoch: 81 [300/12000 (2%)]	Loss: 0.375773
Train Epoch: 81 [600/12000 (5%)]	Loss: 0.292130
Train Epoch: 81 [900/12000 (8%)]	Loss: 0.285533
Train Epoch: 81 [1200/12000 (10%)]	Loss: 0.362083
Train Epoch: 81 [1500/12000 (12%)]	Loss: 0.346101
Train Epoch: 81 [1800/12000 (15%)]	Loss: 0.392972
Train Epoch: 81 [2100/12000 (18%)]	Loss: 0.242929
Train Epoch: 81 [2400/12000 (20%)]	Loss: 0.425091
Train Epoch: 81 [2700/12000 (22%)]	Loss: 0.280758
Train Epoch: 81 [3000/12000 (25%)]	Loss: 0.301709
Train Epoch: 81 [3300/12000 (28%)]	Loss: 0.274021
Train Epoch: 81 [3600/12000 (30%)]	Loss: 0.338547
Train Epoch: 81 [3900/12000 (32%)]	Loss: 0.381845
Train Epoch: 8

Train Epoch: 84 [9000/12000 (75%)]	Loss: 0.388216
Train Epoch: 84 [9300/12000 (78%)]	Loss: 0.252328
Train Epoch: 84 [9600/12000 (80%)]	Loss: 0.416519
Train Epoch: 84 [9900/12000 (82%)]	Loss: 0.368546
Train Epoch: 84 [10200/12000 (85%)]	Loss: 0.390675
Train Epoch: 84 [10500/12000 (88%)]	Loss: 0.368503
Train Epoch: 84 [10800/12000 (90%)]	Loss: 0.289907
Train Epoch: 84 [11100/12000 (92%)]	Loss: 0.306985
Train Epoch: 84 [11400/12000 (95%)]	Loss: 0.284872
Train Epoch: 84 [11700/12000 (98%)]	Loss: 0.291937
====> Epoch: 84 Average train loss: 0.3426
====> Epoch: 84 Average test loss: 0.4377
====> Epoch: 84 Average hit rate in 10 candidate:  0.5364
Train Epoch: 85 [0/12000 (0%)]	Loss: 0.297028
Train Epoch: 85 [300/12000 (2%)]	Loss: 0.291981
Train Epoch: 85 [600/12000 (5%)]	Loss: 0.378828
Train Epoch: 85 [900/12000 (8%)]	Loss: 0.281525
Train Epoch: 85 [1200/12000 (10%)]	Loss: 0.435626
Train Epoch: 85 [1500/12000 (12%)]	Loss: 0.290666
Train Epoch: 85 [1800/12000 (15%)]	Loss: 0.326839
Train Epoch

Train Epoch: 88 [6900/12000 (58%)]	Loss: 0.228416
Train Epoch: 88 [7200/12000 (60%)]	Loss: 0.392318
Train Epoch: 88 [7500/12000 (62%)]	Loss: 0.365430
Train Epoch: 88 [7800/12000 (65%)]	Loss: 0.357419
Train Epoch: 88 [8100/12000 (68%)]	Loss: 0.363933
Train Epoch: 88 [8400/12000 (70%)]	Loss: 0.324950
Train Epoch: 88 [8700/12000 (72%)]	Loss: 0.341724
Train Epoch: 88 [9000/12000 (75%)]	Loss: 0.435052
Train Epoch: 88 [9300/12000 (78%)]	Loss: 0.379535
Train Epoch: 88 [9600/12000 (80%)]	Loss: 0.281299
Train Epoch: 88 [9900/12000 (82%)]	Loss: 0.282061
Train Epoch: 88 [10200/12000 (85%)]	Loss: 0.358470
Train Epoch: 88 [10500/12000 (88%)]	Loss: 0.392208
Train Epoch: 88 [10800/12000 (90%)]	Loss: 0.373248
Train Epoch: 88 [11100/12000 (92%)]	Loss: 0.423581
Train Epoch: 88 [11400/12000 (95%)]	Loss: 0.336683
Train Epoch: 88 [11700/12000 (98%)]	Loss: 0.383396
====> Epoch: 88 Average train loss: 0.3427
====> Epoch: 88 Average test loss: 0.4385
====> Epoch: 88 Average hit rate in 10 candidate:  0.4636
T

Train Epoch: 92 [4800/12000 (40%)]	Loss: 0.264327
Train Epoch: 92 [5100/12000 (42%)]	Loss: 0.244833
Train Epoch: 92 [5400/12000 (45%)]	Loss: 0.320847
Train Epoch: 92 [5700/12000 (48%)]	Loss: 0.357107
Train Epoch: 92 [6000/12000 (50%)]	Loss: 0.304198
Train Epoch: 92 [6300/12000 (52%)]	Loss: 0.303357
Train Epoch: 92 [6600/12000 (55%)]	Loss: 0.300531
Train Epoch: 92 [6900/12000 (58%)]	Loss: 0.416888
Train Epoch: 92 [7200/12000 (60%)]	Loss: 0.344311
Train Epoch: 92 [7500/12000 (62%)]	Loss: 0.401044
Train Epoch: 92 [7800/12000 (65%)]	Loss: 0.355773
Train Epoch: 92 [8100/12000 (68%)]	Loss: 0.391309
Train Epoch: 92 [8400/12000 (70%)]	Loss: 0.497158
Train Epoch: 92 [8700/12000 (72%)]	Loss: 0.334873
Train Epoch: 92 [9000/12000 (75%)]	Loss: 0.336811
Train Epoch: 92 [9300/12000 (78%)]	Loss: 0.379502
Train Epoch: 92 [9600/12000 (80%)]	Loss: 0.280600
Train Epoch: 92 [9900/12000 (82%)]	Loss: 0.301102
Train Epoch: 92 [10200/12000 (85%)]	Loss: 0.320786
Train Epoch: 92 [10500/12000 (88%)]	Loss: 0.30476

Train Epoch: 96 [2700/12000 (22%)]	Loss: 0.288284
Train Epoch: 96 [3000/12000 (25%)]	Loss: 0.294723
Train Epoch: 96 [3300/12000 (28%)]	Loss: 0.379237
Train Epoch: 96 [3600/12000 (30%)]	Loss: 0.353313
Train Epoch: 96 [3900/12000 (32%)]	Loss: 0.336189
Train Epoch: 96 [4200/12000 (35%)]	Loss: 0.318220
Train Epoch: 96 [4500/12000 (38%)]	Loss: 0.317825
Train Epoch: 96 [4800/12000 (40%)]	Loss: 0.265037
Train Epoch: 96 [5100/12000 (42%)]	Loss: 0.357847
Train Epoch: 96 [5400/12000 (45%)]	Loss: 0.356509
Train Epoch: 96 [5700/12000 (48%)]	Loss: 0.294494
Train Epoch: 96 [6000/12000 (50%)]	Loss: 0.371584
Train Epoch: 96 [6300/12000 (52%)]	Loss: 0.388085
Train Epoch: 96 [6600/12000 (55%)]	Loss: 0.364666
Train Epoch: 96 [6900/12000 (58%)]	Loss: 0.389621
Train Epoch: 96 [7200/12000 (60%)]	Loss: 0.426147
Train Epoch: 96 [7500/12000 (62%)]	Loss: 0.294003
Train Epoch: 96 [7800/12000 (65%)]	Loss: 0.300154
Train Epoch: 96 [8100/12000 (68%)]	Loss: 0.289999
Train Epoch: 96 [8400/12000 (70%)]	Loss: 0.305672


## build rnn model to predict secondary metric
test the result in TS_rnn